Pre-installed library imports

In [2]:

import importlib
import subprocess
import sys
import datetime as time

Function to check if the libraries used in this code are already installed

In [3]:
def checkIfInstalled(library):
    try:
        importlib.import_module(library)
        print(f'{library} is installed')
    
    except ImportError:
        print(f'{library} is not installed, installing...')
        
        # Using sys.executable to install the library in the same python version, and -m to run the module as a script and ensure the correct version is used
        subprocess.run([sys.executable,'-m', 'pip', 'install', library])

Aditional library imports 

In [4]:
# Requests library, used to make HTTP requests
checkIfInstalled('requests')
import requests

requests is installed


TBA API setup

In [5]:
url = "https://www.thebluealliance.com/api/v3"
header = {"X-TBA-Auth-Key":"E5xeD6sWIPJe5QDAgtoXzoHkQpKxaoOJlyGXThQBH4SztWmnfEgSbjYLFbC40r1E"}

# Availability check
testResponse = requests.get(url + "/status", headers=header)

print(testResponse.status_code)

200


Fetch individual team data 
    -fetchRecursive: Recursively iterates through a list of dictionaries containing team data checking coincidences on a given team number, if there's a match, a dictionary containing team data is returned, else, an empty list is returned

    -fetchTeamData: Recursively extracts team lists from TBA API and uses fetchRecursive to find a team's data given a team number

FRC team class 

In [6]:
class Team:
    # Static Methods
    @staticmethod
    def fetchRecursive(teamsJson, teamNumber):
        if len(teamsJson) == 0:
            return []
        else:
            if (teamNumber == teamsJson[0]["team_number"]):
                return teamsJson[0] # Returns team dictionary
            else:
                return Team.fetchRecursive(teamsJson[1:], teamNumber)
               
    @staticmethod
    def fetchTeamData(teamNumber, page = 0):
        teamsJson = requests.get(url + f"/teams/2023/{page}/simple", headers=header).json()
    
        if len(teamsJson) == 0:
            return []
        else:
            out = Team.fetchRecursive(teamsJson, teamNumber)
        if type(out) == dict:
            return out 
        else:
            return Team.fetchTeamData(teamNumber, page + 1)
        
    # Class Methods    
    @classmethod
    def instanceByTeamNumber(cls, teamNumber):
        return Team(Team.fetchTeamData(teamNumber))

    # Constructor
    def __init__(self, teamDict):
        # Private atrributes
        self.__teamKey = teamDict["key"]
        
        # Public attributes
        self.teamNumber = teamDict["team_number"]
        self.name = teamDict["name"]
        self.city = teamDict["city"]
        self.state = teamDict["state_prov"]
        self.country = teamDict["country"]
        
    @property
    def teamKey(self):
        return self.__teamKey
    
    def __repr__(self):
        return f"Team {self.teamNumber}"

team4403 = Team.instanceByTeamNumber(4403)

print(team4403.__dict__)

{'_Team__teamKey': 'frc4403', 'teamNumber': 4403, 'name': 'PEÑOLES&PrepaTec Laguna', 'city': 'Torreon', 'state': 'Coahuila', 'country': 'Mexico'}


Event, Match and Alliance Classes

In [7]:
class Event:
    # Static Methods 
    @staticmethod
    def setDate(dateString):
        return time.datetime.strptime(dateString, "%Y-%m-%d").date()
    # Class Methods
    # Constructor
    def __init__(self, eventDict):
        self.__eventKey = eventDict["key"]
        self.__eventCode = eventDict["event_code"]
        self.__eventType = eventDict["event_type"]
        
        self.name = eventDict["name"]
        self.city = eventDict["city"]
        self.state = eventDict["state_prov"]
        self.country = eventDict["country"]
        
        self.startDate = Event.setDate(eventDict["start_date"])
        self.endDate = Event.setDate(eventDict["end_date"])
        
        self.year = int(eventDict["year"])
        self.week = int(eventDict["week"])
        
    
    @property
    def eventKey(self):
        return self.__eventKey
    
    @property
    def eventCode(self):
        return self.__eventCode
    
    @property
    def eventType(self):
        return self.__eventType
        

Gonna make a unique class for managing API endpoints, function tests below

Get event keys, as well as event locations for a given year 

/events/{year}/simple output JSON:
[
  {
    "key": "string",
    "name": "string",
    "event_code": "string",
    "event_type": 0,
    "district": {
      "abbreviation": "string",
      "display_name": "string",
      "key": "string",
      "year": 0
    },
    "city": "string",
    "state_prov": "string",
    "country": "string",
    "start_date": "2024-01-10",
    "end_date": "2024-01-10",
    "year": 0
  }
]

In [29]:
def fetchEvents_inCountry(year, coutry, printData = False):
    eventList = requests.get(url + f"/events/{year}/simple", headers = header).json()
    
    events_inCountry = []
    
    for event in eventList:
        if (coutry.lower() == event["country"].lower()):
            events_inCountry.append(event)

    
    if printData:
        for event in events_inCountry:
            print(event)
    else:
        return events_inCountry
    
fetchEvents_inCountry(2023, 'Mexico', True)


{'city': 'Santa Catarina', 'country': 'Mexico', 'district': None, 'end_date': '2023-09-07', 'event_code': 'mexas', 'event_type': 99, 'key': '2023mexas', 'name': 'Mexico ALL-STAR', 'start_date': '2023-09-05', 'state_prov': 'N.L.', 'year': 2023}
{'city': 'Monterrey', 'country': 'Mexico', 'district': None, 'end_date': '2023-03-04', 'event_code': 'mxmo', 'event_type': 0, 'key': '2023mxmo', 'name': 'Regional Monterrey', 'start_date': '2023-03-01', 'state_prov': 'N.L.', 'year': 2023}
{'city': 'Heroica Puebla de Zaragoza', 'country': 'Mexico', 'district': None, 'end_date': '2023-03-18', 'event_code': 'mxpu', 'event_type': 0, 'key': '2023mxpu', 'name': 'Regional Puebla', 'start_date': '2023-03-15', 'state_prov': 'Pue.', 'year': 2023}
{'city': 'Torreón', 'country': 'Mexico', 'district': None, 'end_date': '2023-03-25', 'event_code': 'mxto', 'event_type': 0, 'key': '2023mxto', 'name': 'Regional Laguna', 'start_date': '2023-03-22', 'state_prov': 'Coah.', 'year': 2023}
200


In [41]:
import TBAmanager as tba

mytba = tba.TBAmanager("E5xeD6sWIPJe5QDAgtoXzoHkQpKxaoOJlyGXThQBH4SztWmnfEgSbjYLFbC40r1E")

print(mytba.listEvents_inCountry(2023, 'Mexico'))

None
